## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
# File location and type
file_location = "/FileStore/tables/data_officer.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df.limit(10))

id,gender,race,appointed_date,rank,active,birth_year,first_name,last_name,tags,middle_initial,suffix_name,resignation_date,complaint_percentile,middle_initial2,civilian_allegation_percentile,honorable_mention_percentile,internal_allegation_percentile,trr_percentile
3728,M,Black,1966-12-19,Lieutenant,No,1945,Jake,Cameron,{},B,JR,1995-02-16,54.9532,null,null,0.0000,null,null
4316,M,White,1965-03-01,Police Officer,No,1934,Donald,Centnarowicz,{},E,null,1995-03-16,0.0000,null,null,0.0000,null,null
4526,M,White,1961-02-13,Detective,No,1935,Daniel,Chilla,{},S,null,1995-06-07,0.0000,null,null,0.0000,null,null
5039,M,White,1967-11-20,Sergeant,No,1937,Dennis,Collins,{},F,null,1995-02-26,54.8532,null,null,0.0000,null,null
5238,M,White,1966-02-28,Police Officer,No,1939,Carmen,Consolino,{},A,null,1995-04-13,35.6277,null,null,0.0000,null,null
6287,M,Black,1962-08-13,Police Officer,No,1936,Raymond,Davis,{},C,null,1995-03-10,35.7027,null,null,0.0000,null,null
6389,M,Black,1968-10-28,Police Officer,No,1946,William,Davis,{},H,JR,1995-06-02,68.0262,null,null,0.0000,null,null
6888,M,White,1966-02-07,Police Officer,No,1936,Philip,Dienethal,{},A,null,1995-02-16,46.4085,null,null,0.0000,null,null
6940,M,White,1966-06-01,Police Officer,No,1945,Donald,Dimberg,{},W,null,1995-07-16,54.4535,null,null,0.0000,null,null
7317,M,White,1962-11-26,Police Officer,No,1931,Dennis,Doyle,{},T,null,1995-07-01,46.0962,null,null,0.0000,null,null


In [3]:
# Create a view or table

temp_table_name = "data_officer"

df.createOrReplaceTempView(temp_table_name)

In [4]:
%sql

WITH gender_loa AS
  (SELECT
    gender, race,
    CASE 
      WHEN active='No' THEN year(resignation_date) - year(appointed_date) 
      ELSE 2018 - year(appointed_date) 
    END as length_of_appointment 
  FROM data_officer 
  WHERE gender = 'M' and race='Black')
SELECT
  CASE
    when length_of_appointment between 0 and 10 then '0-10'
    when length_of_appointment between 10 and 20 then '10-20'
    when length_of_appointment between 20 and 30 then '20-30'
    when length_of_appointment between 30 and 40 then '30-40'
    when length_of_appointment between 40 and 50 then '40-50'
    else '50+'
  END as Range,
  Count(*)
FROM gender_loa
GROUP BY Range
ORDER BY Range ASC



Range,count(1)
0-10,1000
10-20,1177
20-30,2355
30-40,1042
40-50,9
50+,88


In [5]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import year

def encode_race(race):
  if race == 'White':
    return 1
  if race == 'Black':
    return 2
  if race == 'Hispanic':
    return 3
  if race == 'Asian/Pacific':
    return 4
  else:
    return 5

def encode_gender(gender):
  if gender == 'M':
    return 0
  else:
    return 1

udf_encode_race = udf(encode_race, IntegerType())
df = df.withColumn("encoded_race", udf_encode_race("race"))
udf_encode_race = udf(encode_gender, IntegerType())
df = df.withColumn("encoded_gender", udf_encode_race("gender"))
df = df.withColumn("appointment length", year("resignation_date") - year("appointed_date"))
# df.select("encoded_race", "race").show()

In [6]:
print df.stat.corr("encoded_race", "appointment length")
print df.stat.corr("encoded_gender", "appointment length")

-0.307719580124
-0.202160093275